In [84]:
raw_ids = []
raw_texts = []
raw_images = []
with open('./MVSA_Single/labelResultAll.txt', 'r', encoding='ISO-8859-1') as file:
    for line in file:
        text_list = line.replace('\n','').split('	')
        text_label_result= text_list[1].split(',')[0]

        image_label_result = text_list[1].split(',')[1]

        # if not ((text_label_result=='positive' and image_label_result=='negative') 
        #         or (text_label_result=='negative' and image_label_result=='positive')):
        raw_ids.append(text_list[0])
        raw_texts.append(text_label_result)
        raw_images.append(image_label_result)
raw_ids.pop(0)
raw_texts.pop(0)
raw_images.pop(0)

'image'

In [85]:
import pandas as pd
dataset = {'ID':raw_ids, 'Text':raw_texts, 'Image':raw_images}
df = pd.DataFrame(dataset)
# df.to_csv('output_single.csv', index=False)
df

,ID,Text,Image
0,1,neutral,positive
1,2,neutral,positive
2,3,neutral,positive
3,4,positive,positive
4,5,positive,positive
...,...,...,...
4864,5125,neutral,positive
4865,5126,positive,neutral
4866,5127,positive,positive
4867,5128,neutral,positive


In [1]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout
from keras.metrics import Recall,Precision

c:\Users\Jeffan\.conda\envs\capstone2\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  warnings.warn(


In [4]:
train_data_path = './MVSA_Single/train'
val_data_path = './MVSA_Single/val'
test_data_path = './MVSA_Single/test'
weights_path = './weight/vgg16-places365_weights_tf_dim_ordering_tf_kernels.h5'
input_shape = (224, 224, 3)

In [7]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_path,
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical',
        shuffle=False)

val_generator = val_datagen.flow_from_directory(
        val_data_path,
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical',
        shuffle=False)

# Evaluate the model on the test set
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_data_path,
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical',
        shuffle=False)

Found 2415 images belonging to 3 classes.
Found 303 images belonging to 3 classes.
Found 480 images belonging to 3 classes.


In [174]:
train_generator.class_indices

{'negative': 0, 'neutral': 1, 'positive': 2}

In [157]:
image_ids = val_generator.filenames

In [158]:
def getLabel(id):
    text_label = df[df['ID']==id]['Text'].values[0]
    image_label = df[df['ID']==id]['Image'].values[0]
    tag = 0
    label = 'neutral'
    if (text_label=='positive' and image_label=='negative') or (text_label=='negative' and image_label=='positive'):
        label = 'unknown'
    elif (text_label=='positive' and image_label=='neutral') or (text_label=='neutral' and image_label=='positive') or (text_label=='positive' and image_label=='positive'):
        label = 'positive'
    elif (text_label=='negative' and image_label=='neutral') or (text_label=='neutral' and image_label=='negative') or (text_label=='negative' and image_label=='negative'):
        label = 'negative'

    return label

In [159]:
labels = []
ids = []
for filepath in image_ids:
    id = filepath.split('\\')[1].replace('.jpg','')
    label = getLabel(id)
    labels.append(label)
    ids.append(id)
    

In [160]:
texts = []
for id in ids:
    text_path = './MVSA_Single/data/'+id+'.txt'
    with open(text_path, "r", encoding='ISO-8859-1') as f:  # 打开文件
        data = f.read()  # 读取文件
        texts.append(data[0:len(data)-2])

In [161]:
import re
def remove_urls(text):
    url_pattern = re.compile(r'https?://[A-Za-z0-9./]+')
    return url_pattern.sub(r'', text)

texts = list(map(remove_urls, texts))

In [162]:
def remove_tag(text):
    return re.sub(r'#\w*', '', text)
texts = list(map(remove_tag, texts))

In [163]:
def remove_punct(text):
    return re.sub(r'[^\w\s]','',text)
texts = list(map(remove_punct, texts))

In [164]:
def remove_whitespace(text):
    return  " ".join(text.split())
texts = list(map(remove_whitespace, texts))

In [165]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer,word_tokenize
en_stopwords = stopwords.words('english')

def remove_stopwords(text):
   result = []
   text = word_tokenize(text)
   for token in text:
      if token not in en_stopwords:
           result.append(token)
   return ' '.join(result)

texts = list(map(remove_stopwords, texts))
texts

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jeffan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jeffan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['cleethorp_es emmal940 Fifty shades grey',
 'Quantum seas dramatic clouds Notice empty pool cold windy today Tomorrow Florida',
 'Plowin_Donny Am I Maribelle taken aback exactly certain meant tooting',
 '',
 'Frozen lake uni',
 'At',
 'The 21 Deadliest Ani',
 'RT octuse I shipped 5 years wasnt 2 years ago I fell hard two',
 '',
 'We found As I Lay Dying Forsaken audiom',
 'El nuevo trailer de 7 es ciencia ficcin pura',
 'Wai borlog Patiwakal laturrr',
 'ShepherdHomes still comments',
 '',
 '',
 'knowing means feel time',
 'ThatKidRalph I going child violate butttt Im grown',
 'What Jesus teach anger',
 'Really way treat deserve better',
 'night ArenaScreen eOneFilmsUS',
 'Outcast Rooftop Escape',
 'Standing shadows Of sky No one knows Im crying Beaten love',
 'umbrella arrived actually like render shit since colors blotched rn',
 'RT kEsquire The legendary slave burned Montreal Marie Joseph Angelique executed 1734',
 'SeanHMurray whats happening pic',
 'RT MirrorFootball Blackpool got

In [166]:
place_model = VGG16(weights=None, include_top=True, input_shape=input_shape, classes=365)
place_model.load_weights(weights_path)

In [167]:
predictions = place_model.predict_generator(val_generator, steps=len(val_generator), verbose=1)

C:\Users\Jeffan\AppData\Local\Temp\ipykernel_15200\2064706395.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = place_model.predict_generator(val_generator, steps=len(val_generator), verbose=1)


10/10 [==============================] - 27s 3s/step


In [168]:
pred_place_result = []
for pred in predictions:
  pred_place_result.append(np.argsort(pred)[-5:][::-1])

In [169]:
object_model = VGG16(weights=None, include_top=True, input_shape=input_shape)
object_model.load_weights('./weight/vgg16_weights_tf_dim_ordering_tf_kernels.h5')
object_predictions = object_model.predict_generator(val_generator, steps=len(val_generator), verbose=1)

C:\Users\Jeffan\AppData\Local\Temp\ipykernel_15200\1849977555.py:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  object_predictions = object_model.predict_generator(val_generator, steps=len(val_generator), verbose=1)


10/10 [==============================] - 32s 3s/step


In [170]:
pred_result = []

for pred in object_predictions:
    high_confidence_preds = np.where(pred > 0.5)[0]
    sorted_high_confidence_preds = high_confidence_preds[np.argsort(pred[high_confidence_preds])][::-1]
    pred_result.append(sorted_high_confidence_preds)


In [171]:
json_list = []
for id, text, label, top5_preds, object in zip(ids, texts, labels, pred_place_result, pred_result):
        if len(text)>3 and label != 'unknown':
                json_list.append({'id': id, "text":text, "image":'data/image_data/val_image/'+id+'.jpg', "label":label,  'places': top5_preds.tolist(),  "objects":object.tolist()})

In [172]:
import json

with open("./val_all_anno.json", "w") as outfile:
    for obj in json_list:
        json.dump(obj, outfile)
        outfile.write("\n")